In [2]:
import cv2
from tracker import *
import centroidtracker as cent
import numpy as np
import os
import math

tracker = cent.CentroidTracker(maxDisappeared=150, maxDistance=60)

# Reading Video into Python 
cap = cv2.VideoCapture('/Users/gideonowusu/Downloads/PhD 2028/Projects/ML-Melanoma/GitHub/Zoomed_In.avi')

# Get video properties
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define codec and create VideoWriter object
out = cv2.VideoWriter('/Users/gideonowusu/Downloads/PhD 2028/Projects/ML-Melanoma/GitHub/Melanoma Output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

# Object detection from Stable camera
object_detector = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=40)

# Function to calculate Euclidean distance between two points
def calculate_distance(point1, point2):
    x1, y1 = point1
    x2, y2 = point2
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

objects = {}   
maximum_inactive_frames = 2
next_id = 0
some_threshold = 55
line_x = 0
max_inactive_frames = 40
inactive_frames = {}
trajectory_dict = {}

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Preprocessing
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Object Detection
    mask = object_detector.apply(blurred)
    _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    detections = []
    merged_boxes = []
    
    # Process contours
    for cnt in contours:
        contour_area = cv2.contourArea(cnt)
        if 13 < contour_area < 1000:
            x, y, w, h = cv2.boundingRect(cnt)
            centroid = (int(x + w / 2), int(y + h / 2))

            existing_track = None
            for obj_id, bbox in objects.items():
                if calculate_distance(centroid, (bbox[0] + bbox[2] / 2, bbox[1] + bbox[3] / 2)) < some_threshold:
                    existing_track = obj_id
                    break

            if existing_track is not None:
                objects[existing_track] = (x, y, w, h)
                inactive_frames[existing_track] = 0
            else:
                objects[next_id] = (x, y, w, h)
                inactive_frames[next_id] = 0
                next_id += 1
            
            merged = False
            for box in merged_boxes:
                bx, by, bw, bh = box
                if (x >= bx and x <= bx + bw) or (bx >= x and bx <= x + w):
                    if (y >= by and y <= by + bh) or (by >= y and by <= y + h):
                        merged = True
                        merged_boxes.remove(box)
                        xmin = min(x, bx)
                        ymin = min(y, by)
                        xmax = max(x + w, bx + bw)
                        ymax = max(y + h, by + bh)
                        merged_boxes.append((xmin, ymin, xmax - xmin, ymax - ymin))
                        break
            
            if not merged:
                merged_boxes.append((x, y, w, h))
    
    # Draw merged bounding boxes
    for box in merged_boxes:
        x, y, w, h = box
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 1)
    
    objects = tracker.update(detections)
    for (objectId, bbox) in objects.items():
        x1, y1, x2, y2 = bbox
        cX = int((x1 + x1 + x2) // 2.0)
        cY = int((y1 + y1 + y2) // 2.0)
    
        if objectId not in trajectory_dict:
            trajectory_dict[objectId] = [(cX, cY)]
        else:
            trajectory_dict[objectId].append((cX, cY))
    
        text = "ID:{}".format(objectId)
        cv2.putText(frame, text, (x1, y1 - 5), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 0, 255), 1)
        cv2.line(frame, (line_x, 950), (line_x, 10), (0, 255, 0), thickness=1)
    
    # Write frame to output video
    out.write(frame)
    
    # Remove inactive tracks
    inactive_tracks = [obj_id for obj_id, inactive_frame_count in inactive_frames.items() if inactive_frame_count >= max_inactive_frames]
    for obj_id in inactive_tracks:
        del objects[obj_id]
        del inactive_frames[obj_id]

    for obj_id in objects.keys():
        inactive_frames[obj_id] += 1
    
    cv2.imshow("Frame", frame)
    
    key = cv2.waitKey(30)
    if key == 27:
        break

# Release video writer
out.release()
cap.release()
cv2.destroyAllWindows()


2025-02-05 22:05:56.512 python[25268:43390430] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-05 22:05:56.512 python[25268:43390430] +[IMKInputSession subclass]: chose IMKInputSession_Modern
